In [12]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split

In [13]:
try:
    df_trabajo_fusion = pd.read_csv("../output/Datos_Fusionadosv2_escalado.csv")

except Exception as e:
    print(f"Error al cargar los datos: {str(e)}")

In [14]:
# Seleccionar las características y la variable objetivo
X = df_trabajo_fusion.drop(columns=['Tipo_Mantenimiento'])
y = df_trabajo_fusion['Tipo_Mantenimiento']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar los modelos
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'RandomForestRegressor': RandomForestRegressor()
}

# Entrenar y evaluar cada modelo
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    results[name] = mse

# Imprimir los resultados
for name, mse in results.items():
    print(f'{name}: Mean Squared Error = {mse}')

LinearRegression: Mean Squared Error = 0.24186536653700585
Ridge: Mean Squared Error = 0.24185776801167233
Lasso: Mean Squared Error = 0.24930764859318436
RandomForestRegressor: Mean Squared Error = 0.09494746358613762
